In [1]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

sns.set_style('whitegrid')

## Regression models

First let's load in the data:

In [2]:
bikeshare = pd.read_csv('../EDA/bikeshare_train_data.csv', parse_dates=['Date'],index_col=['Date'])

In [3]:
bikeshare

,day_length,min_temp,max_temp,mean_temp,temp_diff,hdd,cdd,rain,snow,total_precip,snow_on_ground,max_gust,mean_dep_temp,mean_ret_temp,mean_ride_temp,total_dist,total_duration,ebike_trips,num_trips,adj_num_trips
Date,,,,,,,,,,,,,,,,,,,,
2017-01-01,497.133333,-2.3,2.2,-0.1,4.5,18.1,0.0,0.0,0.0,0.0,3.0,10.0,3.555556,4.296296,3.925926,338025.00,232693.0,0.0,162,1480.333252
2017-01-02,498.133333,-6.0,1.4,-2.3,7.4,20.3,0.0,0.0,0.0,0.0,2.0,0.0,2.718519,3.688889,3.203704,660054.00,382729.0,0.0,270,1588.333252
2017-01-03,499.133333,-7.8,0.4,-3.7,8.2,21.7,0.0,0.0,0.0,0.0,1.0,0.0,0.807292,1.791667,1.299479,635395.00,376013.0,0.0,384,1702.333252
2017-01-04,501.133333,-8.4,2.2,-3.1,10.6,21.1,0.0,0.0,0.0,0.0,1.0,1.0,2.767391,3.643478,3.205435,766082.00,376721.0,0.0,460,1778.333252
2017-01-05,502.133333,-6.6,0.7,-3.0,7.3,21.0,0.0,0.0,0.0,0.0,1.0,0.0,2.601145,3.524809,3.062977,888222.00,524933.0,0.0,524,1842.333252
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-04-26,861.233333,7.9,14.6,11.3,6.7,6.7,0.0,1.4,0.0,1.4,0.0,0.0,14.621886,14.964365,14.793125,7939371.68,3583575.0,905.0,3171,2560.705662
2023-04-27,865.233333,4.7,14.8,9.8,10.1,8.2,0.0,0.0,0.0,0.0,0.0,0.0,17.535425,17.358553,17.446989,11360210.66,4637628.0,1089.0,3952,3336.736184
2023-04-28,868.233333,5.6,17.0,11.3,11.4,6.7,0.0,0.0,0.0,0.0,0.0,0.0,20.708121,20.481930,20.595026,13985654.33,6220574.0,1265.0,4704,4083.757355


In [ ]:
## More here...